In [ ]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!java -version

In [ ]:
!pip install pyspark

In [ ]:
!git clone https://github.com/leonardoamorim/aulapython.git

In [ ]:
! ls aulapython

In [ ]:
! gunzip aulapython/NASA_access_log_Jul95.gz
! gunzip aulapython/NASA_access_log_Aug95.gz

In [ ]:
! du -h aulapython/NASA_access_log_Jul95

In [ ]:
! du -h aulapython/NASA_access_log_Aug95

In [ ]:
! wc -l aulapython/NASA_access_log_Jul95
! wc -l aulapython/NASA_access_log_Aug95

In [ ]:
from pyspark import SparkConf, SparkContext
from operator import add

In [ ]:
configuracao = (SparkConf().setMaster("local").setAppName("Exercicio")
                .set("spark.executor.memory", "5g"))

In [ ]:
sc = SparkContext(conf = configuracao)

In [ ]:
julho = sc.textFile('aulapython/NASA_access_log_Jul95')
agosto = sc.textFile('aulapython/NASA_access_log_Aug95')

In [ ]:
julho = julho.cache()
agosto = agosto.cache()

In [ ]:
! head -n3 aulapython/NASA_access_log_Jul95

In [ ]:
exemplo = '199.72.81.55 - - [01/Jul/1995:00:00:01 -0400] "GET /history/apollo/ HTTP/1.0" 200 6245'

In [ ]:
operacao = exemplo.split()

In [ ]:
print(operacao)

In [ ]:
print(operacao[-1])

In [ ]:
# Numero de hosts distintos no mes de Julho
contagem_julho = julho.map(lambda line: line.split(' ')[0]).distinct().count()
print("Numero de hosts distintos no mes de Julho:", contagem_julho)

In [ ]:
# Numero de hosts distintos no mes de Agosto
contagem_agosto = agosto.map(lambda line: line.split(' ')[0]).distinct().count()
print("Numero de hosts distintos no mes de Agosto:", contagem_agosto)

In [ ]:
# Funcao para verificar se em uma linha existe um codigohttp igual a 404
def codigo404(linha):
    try:
        codigohttp = linha.split(' ')[-2]
        if codigohttp == '404':
            return True
    except:
        pass
    return False

In [ ]:
erros404_julho = julho.filter(codigo404).cache()
erros404_agosto = agosto.filter(lambda linha: linha.split(' ')[-2] == '404').cache()

In [ ]:
print('Erros 404 em Julho: %s' % erros404_julho.count())

In [ ]:
print('Erros 404 em Agosto: %s' % erros404_agosto.count())

In [ ]:
print(exemplo)

In [ ]:
print(exemplo.split('"'))

In [ ]:
print(exemplo.split('"')[1].split(' ')[1])

In [ ]:
# Funcao para retornar os top5 urls com mais erros 404
def top5_hosts404(rdd):
    urls = rdd.map(lambda linha: linha.split('"')[1].split(' ')[1])
    counts = urls.map(lambda urls: (urls, 1)).reduceByKey(add)
    top5 = counts.sortBy(lambda par: -par[1]).take(5)
    
   # print('\nTop 5 urls:')
   # for urls, count in top5:
   #     print(urls, count)
        
    return top5

In [ ]:
top5_hosts404(erros404_julho)

In [ ]:
top5_hosts404(erros404_agosto)

In [ ]:
print(exemplo)

In [ ]:
print(exemplo.split('[')[1])

In [ ]:
print(exemplo.split('[')[1].split(':')[0])

In [ ]:
# Quantidade de erros 404 por dia
def contador_dias_404(rdd):
    dias = rdd.map(lambda linha: linha.split('[')[1].split(':')[0])
    counts = dias.map(lambda dia: (dia, 1)).reduceByKey(add).collect()
    return counts

In [ ]:
contador_dias_404(erros404_julho)

In [ ]:
contador_dias_404(erros404_agosto)

In [ ]:
# Quantidade de bytes acumulados 
def quantidade_bytes_acumulados(rdd):
    def contador(linha):
        try:
            count = int(linha.split(" ")[-1])
            if count < 0:
                raise ValueError()
            return count
        except:
            return 0
        
    count = rdd.map(contador).reduce(add)
    return count

In [ ]:
print('Quantidade de bytes total em Julho: %s' % quantidade_bytes_acumulados(julho))

In [ ]:
print('Quantidade de bytes total em Agosto: %s' % quantidade_bytes_acumulados(agosto))

In [ ]:
sc.stop()